In [26]:
import pickle
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [27]:
def text_to_bow(path, max_n_gram=2, max_features=2000):
    dataset = pd.read_csv(path)

    with open('data/general/estonian-stopwords.txt') as file:
        lines = [line.rstrip() for line in file]
    stopwords_est = lines
    stop_words = stopwords_est

    CountVec = TfidfVectorizer(ngram_range=(1,max_n_gram), stop_words=stop_words, max_features=max_features)
    # to use bigrams ngram_range=(2,2)
    Count_data = CountVec.fit_transform(dataset.text_features)
    #create dataframe
    bow=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names_out())

    bow = bow.add_prefix('word_')
    bow.index = dataset.index
    return bow

In [28]:
models = {}

dir = 'models/column/'
models['column'] = pickle.load(file = open(dir + 'best_model_rfc_column_2023-03-24_13-50-23_0.6769557425091597.pkl', 'rb'))
dir = 'models/text/'
models['text'] = pickle.load(file = open(dir + 'best_model_rfc_text_2023-03-24_12-41-47_0.5694205848621395.pkl', 'rb'))


data = {}
data['column'] = pd.read_csv('data/inference/AM_ETMM_column.csv')
data['text'] = text_to_bow(path='data/inference/AM_ETMM_text.csv')

/home/till/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.3 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/till/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.3 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/till/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aidu', '

In [29]:
results = pd.DataFrame()
results['id'] = data['column'].index
results.set_index('id', inplace=True)

for key, model in models.items():
    results[key] = [-1] * len(results)

results['safe'] = [-1] * len(results)

In [30]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

for key, model in models.items():
    current_data = data[key]
    print(len(model.feature_names_in_))
    model_features = model.feature_names_in_
    data_features = current_data.columns

    # align features
    model_not_data = [feature for feature in model_features if feature not in data_features]
    current_data[model_not_data] = 0
    intersection = [feature for feature in current_data.columns if feature in model_features]
    current_data = current_data[intersection]
    


853
853
2000
2000
